Skull stripping using convolutional neural networks

Raw image

In [1]:
%matplotlib inline
import os
from helpers import *

import ants
import SimpleITK as sitk

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

AntsPy version = 0.5.4
SimpleITK version = 2.5.0


In [2]:
BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
print(f'project folder = {BASE_DIR}')

project folder = c:\Users\CHEEZYJEEZY\Desktop\MRIdata_preprocessing


In [3]:
raw_examples = [
    'fsl-open-dev_sub-001_T1w.nii.gz',
    'wash-120_sub-001_T1w.nii.gz',
    'kf-panda_sub-01_ses-3T_T1w.nii.gz',
    'listen-task_sub-UTS01_ses-1_T1w.nii.gz'
]

raw_example = raw_examples[3]

In [4]:
raw_img_path = os.path.join(BASE_DIR, 'assets', 'raw_examples', 'brain-lesion_T1w.nii.gz')
raw_img_ants = ants.image_read(raw_img_path, reorient = 'IAL')

explore_3D_array(raw_img_ants.numpy(), cmap = 'nipy_spectral')

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

Deep Learning based method

Load model via AntsPyNet and predict

In [9]:
%pip install antspynet

  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.13.0-cp312-cp312-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached wrapt-1.17.2-cp312-cp312-win_amd64.whl.metadata (6.5 kB)
  Using cached grpcio-1.71.0-cp312-cp312-win_amd64.whl.metadata (4.0 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached markdown-3.8-py3-none-any.whl.metadata (5.1 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.

  You can safely remove it manually.
  You can safely remove it manually.


In [11]:
%pip install distutils

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement distutils (from versions: none)
ERROR: No matching distribution found for distutils


In [5]:
from antspynet.utilities import brain_extraction

In [6]:
prob_brain_mask = brain_extraction(raw_img_ants,modality='t1', verbose = True)

Brain extraction:  retrieving model weights.
Brain extraction:  retrieving template.
Brain extraction:  normalizing image to the template.
Brain extraction:  prediction and decoding.
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Brain extraction:  renormalize probability mask to native space.


In [7]:
print(prob_brain_mask)

explore_3D_array(prob_brain_mask.numpy())

ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (192, 192, 160)
	 Spacing    : (1.25, 1.25, 1.2)
	 Origin     : (98.1114, -149.1525, -129.5975)
	 Direction  : [-0.  0. -1.  0.  1.  0.  1.  0.  0.]



interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

Generate final mask

In [8]:
brain_mask = ants.get_mask(prob_brain_mask, low_thresh = 0.5)

In [9]:
explore_3D_array_with_mask_contour(raw_img_ants.numpy(), brain_mask.numpy())

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

In [10]:
out_folder = os.path.join(BASE_DIR, 'assets', 'preprocessed')
out_folder = os.path.join(out_folder, raw_example.split('.')[0])
os.makedirs(out_folder, exist_ok = True)

out_filename = add_suffix_to_filename(raw_example, suffix = 'brainmaskbyDL')
out_path = os.path.join(out_folder, out_filename)

print(raw_img_path[len(BASE_DIR) :])
print(out_path[len(BASE_DIR) :])

\assets\raw_examples\brain-lesion_T1w.nii.gz
\assets\preprocessed\listen-task_sub-UTS01_ses-1_T1w\listen-task_sub-UTS01_ses-1_T1w_brainmaskbyDL.nii.gz


In [11]:
brain_mask.to_file(out_path)

Generate brain masked

In [ ]:
masked = ants.mask_image(raw_img_ants, brain_mask)
explore_3D_array(masked.numpy())

interactive(children=(IntSlider(value=95, description='SLICE', max=191), Output()), _dom_classes=('widget-inte…

In [13]:
out_filename = add_suffix_to_filename(raw_example, suffix = 'brainMaskedbyDL')
out_path = os.path.join(out_folder, out_filename)


In [14]:
masked.to_file(out_path)